# Centrality plots and stats

In [2]:
import pickle
import numpy as np
import networkx as nx
import scipy
import matplotlib.pyplot as plt
from itertools import product, permutations
from collections import defaultdict
from copy import copy
import os

import scienceplots

plt.style.use(['science'])

from useful_functions import *

## Load data and transform node indices into labels

In [6]:
all_centralities = {}

for file in os.listdir('CentralityData'):
    
    if 'cent' not in file:
        continue
    
    with open(f'CentralityData/{file}', 'rb') as f:
        
        name = file.split('_')[1].split('.')[0]
        all_centralities[name] = pickle.load(f)

In [9]:
all_centralities_names = {}

for name, rankings in all_centralities.items():
    
    labels = rankings['labels']
    inv_labels = {index: molec for molec, index in labels.items()}
    
    EC = {inv_labels[index]:cent for index, cent in rankings['EC'].items()} 
    HEC = {inv_labels[index]:cent for index, cent in rankings['HEC'].items()} 
    
    all_centralities_names[name] = (EC, HEC)

## Relation with in/out-degree

## Top nodes table

In [16]:
K = 5 
for name, (EC, HEC) in all_centralities_names.items(): 
    print(topN(EC, K).keys(), topN(HEC, K).keys())

dict_keys(['H', 'H2', 'CO', 'C', 'CN']) dict_keys(['E', 'C', 'C+', 'O', 'H+'])
dict_keys(['H', 'H2', 'CO', 'C', 'He']) dict_keys(['e-', 'He+', 'Photon', 'O', 'C'])
dict_keys(['H', 'H2', 'C', 'CO', 'CN']) dict_keys(['e-', 'He+', 'Photon', 'C+', 'H+'])
dict_keys(['H', 'OH', 'O', 'CH3', 'HCO']) dict_keys(['OH', 'H', 'O', 'CH3', 'O2'])
dict_keys(['H2', 'CH4', 'H', 'CH3', 'NO']) dict_keys(['H', 'CH3SH', 'CH3', 'H2CS', 'HNO'])


$$
\begin{table}{ccc}
a & b & c\\
\end{table}
$$

$$
x = 2
$$

## Network centrality visualization